## Create test data

Load all needed data

In [33]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit.shore import ShoreModel, ShoreFit

os.chdir("/home/olivier/Devel/test/samuel/in4michi/")

# Load fractional anisotropy
dti_fa = nib.load("dti_FA.nii.gz")

# Load DTI mask
dti_mask = nib.load("mask.nii.gz")

# Load and adjust tissue segmentation masks
csf_mask = nib.load("fast_pve_0.nii.gz")
gm_mask = nib.load("fast_pve_1.nii.gz")
wm_mask = nib.load("fast_pve_2.nii.gz")

dti_vecs = nib.load("dti_V1.nii.gz")

data = nib.load("complete_data/data.nii.gz")

bvals, bvecs = read_bvals_bvecs("bvals", "bvecs")
gtab = gradient_table(bvals, bvecs)

Specify the voxels you want in the test data

In [34]:
wm_voxels = [(65,42,33),(32,47,31),(43,39,19)]
gm_voxels = [(25,44,25),(50,46,25),(55,38,20)]
csf_voxels = [(49,55,25),(38,68,45),(61,28,45)]
add_voxels = [(61,26,23),(60,49,37),(65,24,37)]

TEST_DATA = "/home/olivier/Devel/test/samuel/in4michi/test_data"

files = {"dti_FA.nii.gz": dti_fa,
         "mask.nii.gz": dti_mask,
         "fast_pve_0.nii.gz": csf_mask,
         "fast_pve_1.nii.gz": gm_mask,
         "fast_pve_2.nii.gz": wm_mask}

Build the test data for the scalar files 

In [35]:
def build_test_data(files, shape):
    for name, img in files.items():
        w = np.array([img.get_data()[wm_voxels[x]] for x in range(2)])
        g = np.array([img.get_data()[gm_voxels[x]] for x in range(2)])
        c = np.array([img.get_data()[csf_voxels[x]] for x in range(2)])
        a = np.array([img.get_data()[add_voxels[x]] for x in range(2)])

        stacked = np.vstack((w, g, c, a)).reshape(shape)
        img_data = nib.Nifti1Image(stacked, img.affine)
        nib.save(img_data, os.path.join(TEST_DATA, name))

In [36]:
build_test_data(files, (2,2,2))
build_test_data({"dti_V1.nii.gz": dti_vecs}, (2,2,2,3))
build_test_data({"data.nii.gz": data}, (2,2,2,97))



## Calculate response result for test data with old code

In [37]:
from subprocess import check_output
from os.path import expanduser
import os

In [40]:
home = expanduser("~")

SCRIPT = 'shore-response.py '
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi/test_data/')
OUT_DIR = os.path.join(DATA_DIR, 'results')
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

command = venv + SCRIPT + '{dire} {out}/response.npz --mask \
{dire}mask.nii.gz'.format(dire=DATA_DIR, out = OUT_DIR)
out = check_output(command, shell = True)

for line in out.splitlines():
    print(line.decode("utf-8") )

order (radial / angular): 4 / 4
Using provided DTI mask.
loading data....
csf response:
  0.   done   
1 voxel
[ 91630.14341921 -90293.49816839 198821.42034613]
grey matter response:
  0.   done   
1 voxel
[ 93831.55649995 -11433.93332202  24600.73636067]
white matter response:
  0.   done   
1 voxel
[ 85887.77198972 -34586.89802036  21162.80444017      0.
      0.              0.        ]


## Calculate deconvolution results for test data with old code

In [41]:
home = expanduser("~")

SCRIPT = 'shore-deconvolve.py '
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi/test_data')
OUT_DIR = os.path.join(home, 'Devel/test/samuel/in4michi/test_data/results')
OPTIONS = ['hpsd', 'nonneg', 'none']
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

for option in OPTIONS:
    command = venv + SCRIPT + '{dire}/data.nii.gz {dire}/response.npz \
{out}/odf{opt}.nrrd --pos {opt} --csf {out}/csf{opt}.nrrd \
--gm {out}/gm{opt}.nrrd --wm {out}/wm{opt}.nrrd'.format(dire=DATA_DIR, opt=option, out=OUT_DIR)
    out = check_output(command, shell = True)
    for line in out.splitlines():
        print(line.decode("utf-8") )

order (radial / angular): 4 / 4
[[281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 [281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 [281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 ...
 [ 93.2258914    0.           0.         ...   0.          82.49415443
   29.13357179]
 [ 93.2258914    0.           0.         ...   0.          82.49415443
   29.13357179]
 [ 93.2258914    0.           0.         ...   0.          82.49415443
   29.13357179]]
Condition number of M: 4.644362534940907e+117
Optimizing...
  0.   done   
order (radial / angular): 4 / 4
[[281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 [281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 [281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 ...
 [ 93.2258914    0.           0.         ...   0.          82.49415

In [8]:
import pyshtools as sh